# Insights
In the following we use the csv relational representation of the sOCEL to explore the data and to get some insights.

In [ ]:
import pandas as pd
import plotly.express as px
import os

from misc.loadCSVocel import get_ocel_df, get_eo_tables

PATH_PREFIX = os.path.join(os.path.abspath(''), 'data\\socel-csv\\')

TABLES = get_ocel_df(PATH_PREFIX)
eventTypeTableFilenames, objectTypeTableFilenames = get_eo_tables(PATH_PREFIX)
TABLES.keys()

### Aggregated Energy consumption (indicator) + aggregated total impact in co2e (score)

In [2]:
joined_result = pd.DataFrame()
for fn in eventTypeTableFilenames+objectTypeTableFilenames:
    table_name = fn.split("\\")[-1].split(".")[0]
    df = TABLES[table_name]
    
    # aggregate over all entries
    results = df.apply(pd.to_numeric, errors='coerce', result_type="expand").sum().dropna()
    results = results.to_frame(name=table_name)
    joined_result = pd.concat([joined_result, results], axis=1, sort=False)
    
joined_result = joined_result.transpose()
joined_result["OCEL Event or Object Type"] = joined_result.index
joined_result = joined_result.set_index("OCEL Event or Object Type")

# only keep index and colums co2e and electricity
joined_result = joined_result[["i_electric-from-grid-de[kWh]", "s_co2e[kg]"]]

#df_vis = df_vis.sort_values(ascending=False)
px.bar(joined_result, 
       x=joined_result.index, 
       y=joined_result.columns, 
       title="Number of rows per table",
       labels={"value": "Aggregated values", "variable": "Impact Attribute"},
       #log_y=True,
       barmode='group',
       height=450)


c:\Users\Marco\anaconda3\envs\env-ids-ws23\lib\site-packages\plotly\express\_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


### Tracking a singel "trace"
As IDs are in some extend inherited from workpiece to workpiece, we can track a single trace of a workpiece through the production process.

In [3]:
import pandas as pd
import plotly.express as px

dataset = TABLES.copy()

date = pd.to_datetime("2023-04-04",  format="%Y-%m-%d").date()

def filter_ocel_id(df):
    if "ocel_id" in df.columns:
        df = df[df["ocel_id"].str.endswith("_1") == True]
    return df

obj_data = pd.DataFrame()
for name, df in dataset.items():
    if "object_" in name and not name in ['object_object', 'object_map_type']:        
        #df = df[df["ocel_time"].dt.date == date]        
        df = df.reset_index()
        df = filter_ocel_id(df)
        df["old_ocel_id"] = df["ocel_id"]
        df = df.drop(["ocel_id"], axis=1)
        df['Dataset'] = name  # Add a column to distinguish datasets
        df["ocel_id"] = df["old_ocel_id"].str.split(">").str[0].str.split("_").str[-1]
        df = df.reset_index()
        obj_data = pd.concat([obj_data, df], ignore_index=True)   
obj_data = obj_data.sort_values(by=['ocel_time'])        

dataset = TABLES.copy()


# Prepare a DataFrame for plotting
ev_data = pd.DataFrame()
for name, df in dataset.items():
    if "event_" in name and not name in ['event_map_type', 'event_object']:        
        df["ocel_time"] = pd.to_datetime(df["ocel_time"], format="%Y-%m-%d %H:%M:%S")
        df = df.reset_index()
        #df = filter_ocel_id(df)
        df["old_ocel_id"] = df["ocel_id"]
        df = df.drop(["ocel_id"], axis=1)
        df['Dataset'] = name  # Add a column to distinguish datasets
        df["ocel_id"] = df["old_ocel_id"].str.split(">").str[0].str.split("_").str[-1]
        df = df.reset_index()
    
        ev_data = pd.concat([ev_data, df], ignore_index=True)
ev_data  = ev_data.sort_values(by=['ocel_time'])

display(obj_data)
display(ev_data)

,level_0,index,ocel_time,ocel_changed_field,m_metadata,old_ocel_id,Dataset,ocel_id,p_mass[kg],p_material[EN10130:2006],s_co2e[kg],i_steel-waste[kg],P_electric-from-grid-de[kWh],P_gas-from-grid-de[m3],p_init-len[cm],p_width[cm],i_material-cold-rolled-steel[kg],i_steel-waste-to-recycle[kg],i_material-steel-pin[kg],P_compressed-air[m3]
0,0,0,2023-04-03 07:30:00,NaN,DE01,o_facility_1,object_Facility,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0,2023-04-03 07:30:00,NaN,NaN,o_steelcoil_1,object_SteelCoil,1,31.91076,DC01,80.734223,NaN,NaN,NaN,4522.5,3.0,31.91076,NaN,NaN,NaN
8,0,0,2023-04-03 07:30:00,NaN,NaN,o_steelpin_1,object_SteelPin,1,0.00100,DC01,0.001950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN
10,0,0,2023-04-03 07:30:00,NaN,NaN,o_worker_1,object_Worker,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,0,0,2023-04-03 07:30:00,NaN,NaN,o_workstation_1,object_Workstation,1,NaN,NaN,NaN,NaN,o_workstation_1,NaN,NaN,NaN,NaN,NaN,NaN,(Pointer-W-A-1)
9,0,0,2023-04-03 07:38:46,NaN,NaN,o_steelsheet_1,object_SteelSheet,1,0.02114,DC01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2023-04-03 07:55:25,NaN,NaN,o_formedpart_1,object_FormedPart,1,NaN,DC01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2,2023-04-03 07:56:00,p_mass[kg],NaN,o_formedpart_1,object_FormedPart,1,0.02114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,0,2023-04-03 09:00:19,NaN,NaN,o_hingepack_1,object_HingePack,1,NaN,DC01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79,79,2023-04-03 09:10:49,NaN,NaN,o_femalepart_1,object_FemalePart,1,NaN,DC01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,level_0,index,ocel_time,p_duration[s],s_co2e[kg],old_ocel_id,Dataset,ocel_id,i_compressed-air[m3],i_electric-from-grid-de[kWh],i_coating-material[kg],i_coating-material-waste[kg],i_gas-n2-used[m3],i_gas-n2-emiited-to-air[m3],i_steel-waste[kg],i_gas_input[Wh],i_emission-of-burn[Wh],i_cardboard-box[kg],i_steel-waste-to-recycle[kg]
32613,0,0,2023-04-03 07:38:46,6.458151,0.006758,e_split_1,event_SplitSteelSheet,1,NaN,0.009867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00042
32614,1,1,2023-04-03 07:38:53,6.669426,0.006978,e_split_2,event_SplitSteelSheet,2,NaN,0.010189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00042
32615,2,2,2023-04-03 07:38:59,6.461274,0.006761,e_split_3,event_SplitSteelSheet,3,NaN,0.009871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00042
32616,3,3,2023-04-03 07:39:06,6.560815,0.006865,e_split_4,event_SplitSteelSheet,4,NaN,0.010023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00042
32617,4,4,2023-04-03 07:39:12,6.529158,0.006832,e_split_5,event_SplitSteelSheet,5,NaN,0.009975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5821,2912,2912,2023-04-06 16:02:25,NaN,0.00142956,e_checkFemale_5275>true,event_CheckFemalePart,5275,0.022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5822,2913,2913,2023-04-06 16:02:35,NaN,0.00077976,e_checkFemale_5465>true,event_CheckFemalePart,5465,0.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2907,2907,2907,2023-04-06 16:02:35,5.000000,NaN,e_assembly_5275,event_AssembleHinge,5275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5823,2914,2914,2023-04-06 16:02:46,NaN,0.00116964,e_checkFemale_5399>true,event_CheckFemalePart,5399,0.018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Distribution of traces by time
As IDs are also created simply by counting up, we can also track a kind-of-distribution of traces by time.

In [4]:
dataset = TABLES.copy()

# Prepare a DataFrame for plotting
plot_data = pd.DataFrame()
for name, df in dataset.items():
    if "event_" in name and not name in ['event_map_type', 'event_object']:        
        df["ocel_time"] = pd.to_datetime(df["ocel_time"], format="%Y-%m-%d %H:%M:%S")
        #df = df[df["ocel_time"].dt.date == date]        
        df = df.reset_index()
        #df = filter_ocel_id(df)
        df["old_ocel_id"] = df["ocel_id"]
        df = df.drop(["ocel_id"], axis=1)
        df['Dataset'] = name  # Add a column to distinguish datasets
        df["ocel_id"] = df["old_ocel_id"].str.split(">").str[0].str.split("_").str[-1]
        df = df.reset_index()
    
        plot_data = pd.concat([plot_data, df], ignore_index=True)
        
df = plot_data

df["ocel_id"] = df["ocel_id"].astype(int)
df.sort_values(by=['ocel_id'], inplace=True)

fig = px.scatter(df, x='ocel_time', y='ocel_id', color='Dataset',
                 title='Scatter Plot of Filtered Datasets',
                 labels={'ocel_id': 'ocel_id', 'ocel_time': 'ocel_time'})
fig.show()

c:\Users\Marco\anaconda3\envs\env-ids-ws23\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

